# Multiple Linear Regression

**Linear Regression** is a statistical method used to model the relationship between a dependent variable and one or more independent variables. The goal of linear tip is to find a linear equation that best predicts the dependent variable from the independent variables.

## Formula

The equation for a linear regression line is:

$$
y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \dots + \beta_nx_n + \epsilon
$$

where:
-  𝑦  is the dependent variable,
-  𝛽_0, 𝛽_1, 𝛽_2, ..., 𝛽_n  are the coefficients,
- 𝑋_1, 𝑋_2, ..., 𝑋_n are the independent variables,
- 𝜖 is the error term, which accounts for the variability in 𝑦 not explained by the independent variables.

## Assumptions

Linear regression assumes:
1. **Linearity:** The relationship between the predictors and the response is linear.
2. **No multicollinearity:** Predictors are not perfectly collinear or highly correlated.
3. **Homoscedasticity:** The variance of residual is the same for any value of the predictors.
4. **Independence:** Observations are independent of each other.
5. **Normality:** For any fixed value of X, Y is normally distributed.

## Types

There are two main types of linear regression:
- **Simple Linear Regression:** Only one independent variable is used to predict the dependent variable. It models the relationship between the dependent variable and one independent variable using a linear equation.
- **Multiple Linear Regression:** More than one independent variable is used to predict the dependent variable.

## Applications

Linear regression is widely used in various fields including economics, biology, engineering, and social sciences to:
- Predict outcomes,
- Determine strength of predictors,
- Forecast future trends,
- Model relationships.

# Implementation

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### Import and show Data

In [2]:
data = pd.read_csv('./Data/BeadArea.csv')
display(data.head())
print ("The data is composed of ", data.shape[0], " rows and ", data.shape[1], " columns.")

,Time,Position Feedback,Velocity Feedback from Axis 1,Velocity Feedback from Axis 2,Temperature,Current Feedback,Interpolated Bead Area
0,0.056,15.567,11.178,6.4952,1.1173,1.2309,1.1184
1,0.057,15.579,11.257,6.4952,1.1175,1.2735,1.1191
2,0.058,15.590,11.375,3.2476,1.1175,1.1567,1.1198
3,0.059,15.601,11.324,6.4952,1.1173,1.2975,1.1205
4,0.060,15.613,11.285,6.4952,1.1173,1.2168,1.1212


The data is composed of  295568  rows and  7  columns.


### Data Preprocessing

In [3]:
data['Lag1'] = data['Interpolated Bead Area'].shift(1)
data['Lag2'] = data['Interpolated Bead Area'].shift(2)
data['Lag3'] = data['Interpolated Bead Area'].shift(3)
data.dropna(inplace=True)

### Predict Bead Area

In [4]:
# Load your dataset
# Assume `data` is your full dataset loaded as a DataFrame
# data = pd.read_csv('your_dataset.csv')  # Replace with your data loading code

# Define widgets with adjusted layout
index_range_slider = widgets.IntRangeSlider(
    value=[0, 50000],
    min=0,
    max=len(data),
    step=1,
    description='Index Range:',
    layout=widgets.Layout(width='600px'),  # Increase width for better readability
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

feature_select = widgets.SelectMultiple(
    options=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    value=['Lag1', 'Lag2', 'Lag3', 'Time', 'Position Feedback', 'Velocity Feedback from Axis 1', 'Velocity Feedback from Axis 2', 'Temperature', 'Current Feedback'],
    description='Features:',
    layout=widgets.Layout(width='600px', height='180px'),  # Increase width and height
    style={'description_width': '150px'},  # Increase description width
    disabled=False
)

train_size_slider = widgets.IntSlider(
    value=80,
    min=50,
    max=95,
    step=1,
    description='Train %:',
    layout=widgets.Layout(width='600px'),  # Increase width
    style={'description_width': '150px'},  # Increase description width
    continuous_update=False
)

apply_button = widgets.Button(description="Apply Changes", layout=widgets.Layout(width='800px'))

# Define the function to apply changes and update the plots
def apply_changes(b):
    with output:
        clear_output(wait=True)
        
        # Extract the parameters from widgets
        index_range = index_range_slider.value
        selected_features = list(feature_select.value)
        train_size_pct = train_size_slider.value / 100
        
        # Slice the data
        df = data[index_range[0]:index_range[1]]
        
        # Prepare the data (assuming 'Interpolated Bead Area' is already in `df`)
        X = df[selected_features]
        y = df['Interpolated Bead Area']
        
        # Train-test split
        train_size = int(len(df) * train_size_pct)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Train the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict on test data
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        display(HTML(f'<b>Mean Squared Error: {mse:.5f}</b>'))  # Display MSE in bold
        
        # Plot predicted vs actual
        plt.figure(figsize=(10, 6))
        plt.plot(y_train.index, y_train, label='Training', color='green')
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.xlabel('Time')
        plt.ylabel('Interpolated Bead Area')
        plt.title('Actual vs Predicted Interpolated Bead Area')
        plt.legend()
        plt.show()
        
        # Calculate loss for each point
        pointwise_mse_loss = (y_test - y_pred) ** 2
        
        # Plot the pointwise loss
        plt.figure(figsize=(10, 6))
        plt.plot(y_test.index, y_test, label='Actual', color='blue')
        plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
        plt.plot(y_test.index, pointwise_mse_loss, label='Pointwise MSE Loss', color='orange')
        plt.xlabel('Time')
        plt.ylabel('MSE Loss')
        plt.title('Pointwise MSE Loss of Predicted vs Actual Interpolated Bead Area')
        plt.legend()
        plt.show()

# Link the apply button to the function
apply_button.on_click(apply_changes)

# Display the widgets and the output area
output = widgets.Output()

display(index_range_slider, feature_select, train_size_slider, apply_button, output)


IntRangeSlider(value=(0, 50000), continuous_update=False, description='Index Range:', layout=Layout(width='600…

SelectMultiple(description='Features:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8), layout=Layout(height='180px', width…

IntSlider(value=80, continuous_update=False, description='Train %:', layout=Layout(width='600px'), max=95, min…

Button(description='Apply Changes', layout=Layout(width='800px'), style=ButtonStyle())

Output()